In [2]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU
from keras.callbacks import CSVLogger
import matplotlib.pyplot as plt

In [3]:
fi = open('keptcolumns.txt', 'r')
keptcolumns = fi.read().rstrip('\n').split(', ')
keptcolumns = np.array(keptcolumns)
data = pd.read_csv('epi_r_toy.csv', usecols = keptcolumns)
# data = pd.read_csv('epi_r.csv', usecols = keptcolumns) # comment out if you want to use the original dataset 
data = data[pd.notnull(data['calories'])]
data = data[pd.notnull(data['protein'])]
data = data[pd.notnull(data['fat'])]
data = data[data['calories'] != 0]
data = data[data['protein'] != 0]
data = data[data['fat'] != 0]
data = data.values
h_val = data[:, 0:6]
ingred = data[:, 6:]

In [4]:
def gen_dat(xy):
    cnt = np.sum(xy == 1)
    idx = np.where(xy == 1)
    x = np.zeros((cnt, xy.shape[0]))
    y = np.zeros((cnt, xy.shape[0]))
    for i in range(cnt):
        y_idx = idx[0][i];
        x_idx = np.setdiff1d(idx[0], y_idx)
        y[i, y_idx] = 1
        x[i, x_idx] = 1
    return x, y

In [5]:
x = np.empty((0, ingred.shape[1]))
y = np.empty((0, ingred.shape[1]))
for i in range(ingred.shape[0]):
    x_tmp, y_tmp = gen_dat(ingred[i])
    x = np.append(x, x_tmp, axis = 0)
    y = np.append(y, y_tmp, axis = 0)
print(x.shape)
print(y.shape)

(8673, 308)
(8673, 308)


In [6]:
idx = np.arange(x.shape[0])
np.random.shuffle(idx)
x = x[idx]
y = y[idx]
p = x.shape[0] * 8 // 10
x_train = x[:p, :]
y_train = y[:p, :]
x_test = x[p:, :]
y_test = y[p:, :]
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(6938, 308)
(6938, 308)
(1735, 308)
(1735, 308)


In [7]:
model1 = Sequential([Dense(200, input_shape=(308,)),
                                    LeakyReLU(),
                                    
                                    Dense(300),
                                    LeakyReLU(),
                                    
                                    Dense(300),
                                    LeakyReLU(),
                                                                                                            
                                    Dense(300),
                                    LeakyReLU(),
                                    
                                    Dense(300),
                                    LeakyReLU(),
                                    
                                    Dense(308, activation='sigmoid')
                                   ])
model1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 200)               61800     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 300)               60300     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 300)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 300)               90300     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 300)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 300)              

In [8]:
csv_logger = CSVLogger('training1.log')
model1.fit(x_train, y_train, epochs = 22, batch_size = 128, callbacks = [csv_logger])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/22
6938/6938 [==============================] - 1s 110us/step - loss: 5.2791 - categorical_accuracy: 0.0241
Epoch 2/22
6938/6938 [==============================] - 1s 83us/step - loss: 4.9686 - categorical_accuracy: 0.0231
Epoch 3/22
6938/6938 [==============================] - 1s 85us/step - loss: 4.7853 - categorical_accuracy: 0.0278
Epoch 4/22
6938/6938 [==============================] - 1s 88us/step - loss: 4.5922 - categorical_accuracy: 0.0329
Epoch 5/22
6938/6938 [==============================] - 1s 100us/step - loss: 4.3737 - categorical_accuracy: 0.0334
Epoch 6/22
6938/6938 [==============================] - 1s 90us/step - loss: 4.1749 - categorical_accuracy: 0.0412
Epoch 7/22
6938/6938 [==============================] - 1s 90us/step - loss: 3.9955 - categorical_accuracy: 0.0470
Epoch 8/22
6938/6938 [==============================] - 1s 92us/step - loss: 3.8207 - categorical_

In [9]:
x = ingred
calories = h_val[:, 2]
protein = h_val[:, 3]
fat = h_val[:, 4]
y = protein / (calories * fat)
print(np.where(y == 0))
idx = np.argsort(y)
idx = idx[:-250]
np.random.shuffle(idx)
x = x[idx]
y = y[idx]
maxi = np.max(y)
y = y / maxi

(array([], dtype=int64),)


In [10]:
model2 = Sequential([Dense(200, input_shape=(308,)),
                                    LeakyReLU(),
                                    
                                    Dense(500),
                                    LeakyReLU(),
                                    
                                    Dense(500),
                                    LeakyReLU(),
                    
                                    Dense(500),
                                    LeakyReLU(),
                    
                                    Dense(500),
                                    LeakyReLU(),
                    
                                    Dense(500),
                                    LeakyReLU(),
                                                        
                                    Dense(1)
                                   ])
model2.compile(optimizer='adam', loss='mean_squared_error')
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 200)               61800     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 200)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 500)               100500    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 500)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 500)               250500    
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 500)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 500)              

In [13]:
csv_logger = CSVLogger('training2.log')
model2.fit(x, y, epochs = 22, batch_size=1024, callbacks=[csv_logger])

Epoch 1/22
1909/1909 [==============================] - 0s 110us/step - loss: 0.0018
Epoch 2/22
1909/1909 [==============================] - 0s 112us/step - loss: 0.0018
Epoch 3/22
1909/1909 [==============================] - 0s 106us/step - loss: 0.0018
Epoch 4/22
1909/1909 [==============================] - 0s 113us/step - loss: 0.0017
Epoch 5/22
1909/1909 [==============================] - 0s 113us/step - loss: 0.0017
Epoch 6/22
1909/1909 [==============================] - 0s 118us/step - loss: 0.0017
Epoch 7/22
1909/1909 [==============================] - 0s 113us/step - loss: 0.0018
Epoch 8/22
1909/1909 [==============================] - 0s 121us/step - loss: 0.0019
Epoch 9/22
1909/1909 [==============================] - 0s 115us/step - loss: 0.0022
Epoch 10/22
1909/1909 [==============================] - 0s 115us/step - loss: 0.0020
Epoch 11/22
1909/1909 [==============================] - 0s 113us/step - loss: 0.0019
Epoch 12/22
1909/1909 [==============================] - 0s 119

In [14]:
cnt1 = 0
cnt2 = 0
for i in range(x_test.shape[0]):
    x = x_test[i].reshape(1, x_test.shape[1])
    y = model1.predict(x, batch_size = 128)
    y_idx = np.argsort(y[0])
    y_idx = y_idx[-20:]
    if np.where(y_test[i] == 1) in y_idx:
        cnt1 += 1
    cnt2 += 1
print(cnt1 / cnt2)

0.5354466858789625


In [15]:
cnt1 = 0
cnt2 = 0
for i in range(x_train.shape[0]):
    x = x_train[i].reshape(1, x_train.shape[1])
    y = model1.predict(x, batch_size = 128)
    y_idx = np.argsort(y[0])
    y_idx = y_idx[-20:]
    if np.where(y_train[i] == 1) in y_idx:
        cnt1 += 1
    cnt2 += 1
print(cnt1 / cnt2)

0.9835687518016719


In [38]:
def test_model(x_idx):
    x = np.zeros((1, ingred.shape[1]))
    x[0, x_idx] = 1
    y = model1.predict(x, batch_size = 128)
    y_idx = np.argsort(y[0])
    y_idx = y_idx[-5:]
    print('Initial Suggestions: ')
    print(keptcolumns[y_idx + 6])
    print('Scores: ')
    print(y[0][y_idx])
    h_val = np.zeros(5)
    for i in range(5):
        x_tmp = np.append(x_idx, y_idx[i])
        x = np.zeros((1, 308))
        x[0, x_tmp] = 1        
        y_tmp = model2.predict(x, batch_size = 128)
        h_val[i] = y_tmp
    y_idx_idx = np.argsort(h_val)
    y_idx = y_idx[y_idx_idx]
    print('Final Suggestions Based on Health: ')
    print(keptcolumns[y_idx + 6])
    print('Health values: ')
    print(h_val[y_idx_idx])

In [39]:
rec_no = 78
print("Recipe name: ", h_val[rec_no][0])
idx = np.array(np.where(ingred[rec_no] == 1))
# print(idx)
print("Original Ingredients: ")
print(keptcolumns[idx + 6])
test_model(np.array([49, 93, 118, 263, 294]));

Recipe name:  Asian Noodles with Barbecued Duck Confit 
Original Ingredients: 
[['carrot' 'duck' 'green bean' 'noodle' 'soy sauce' 'vinegar']]
Initial Suggestions: 
['fennel' 'radish' 'lentil' 'rice' 'noodle']
Scores: 
[0.00085223 0.00102717 0.00204197 0.00388891 0.08549684]
Final Suggestions Based on Health: 
['noodle' 'lentil' 'fennel' 'radish' 'rice']
Health values: 
[0.12778445 0.33051831 0.33354715 0.47515813 0.48013535]
